In [2]:
import os, glob, sys
import matplotlib.pyplot as plt
import numpy as np
import h5py
import tensorflow as tf
%matplotlib inline
import skimage
from skimage.io import imread, imshow, imsave
from tensorflow.python.keras.models import *
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.optimizers import *
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler
# from keras import backend as keras
import time
import functools
from eval import *
from ShowColors import *
from ImportUtil import *
%env CITYSCAPES_DATASET = /home/rvygon/data/
from tensorflow.metrics import *
batch_size = 2
%load_ext autoreload
%autoreload 2

env: CITYSCAPES_DATASET=/home/rvygon/data/


In [3]:
def shifted_image(image, label, width_shift_range, height_shift_range):
    """horisontal or vertical shift"""
    if width_shift_range or height_shift_range:
        if width_shift_range:
            width_shift_range = tf.random_uniform([], -width_shift_range * img_shape[1], width_shift_range * img_shape[1])
        if height_shift_range:
            height_shift_range = tf.random_uniform([], -height_shift_range * img_shape[0], height_shift_range * img_shape[0])
        image = tf.contrib.image.translate(image, [width_shift_range, height_shift_range])
        label = tf.contrib.image.translate(label, [width_shift_range, height_shift_range])
    return image, label

def flip_img(horizontal_flip, image, label):
    if horizontal_flip:
        flip_prob = tf.random_uniform([], 0.0, 1.0)
        image, label = tf.cond(tf.less(flip_prob, 0.5),
                                   lambda: (tf.image.flip_left_right(image), tf.image.flip_left_right(label)),
                                   lambda: (image, label))
    return image, label            

def crop_img(crop_rate, image, label):
    if crop_rate is not None:
        image = tf.image.resize_images(tf.image.central_crop(image, crop_rate), (img_shape[0], img_shape[1]), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        label = tf.image.resize_images(tf.image.central_crop(label, crop_rate), (img_shape[0], img_shape[1]), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image, label

def _augment(image,
             label,
             hue_delta=0,
             horisontal_flip=False,
             width_shift_range=0,
             height_shift_range=0,
             crop_rate=None):
    if hue_delta:
        image = tf.image.random_hue(image, hue_delta)
    image, label = flip_img(horisontal_flip, image, label)
    image, label = shifted_image(image, label, width_shift_range, height_shift_range)
    image, label = crop_img(crop_rate, image, label)
    return image, label
def to_tensor(image, label):
    return image, label

def get_dataset(images, labels,
                preproc_fn=functools.partial(_augment),
                threads=5,
                batch_size=batch_size,
                shuffle=True):
    num_x = len(images)
    features_placeholder = tf.placeholder(images.dtype, images.shape)
    labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

    dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
    dataset = dataset.repeat().batch(batch_size)
   
    #dataset = dataset.map(to_tensor, num_parallel_calls=threads)
   # dataset = dataset.map(preproc_fn)
#     if shuffle:
#         dataset = dataset.shuffle(num_x)
    
   
   

    return dataset

In [5]:
#run this cell once
#%run  cityscapesscripts/preparation/createTrainIdLabelImgs
def upd_print(str):
            sys.stdout.write('\r')       
            sys.stdout.write(str)
            sys.stdout.flush()

In [4]:
img_shape = (256, 512, 3)
batch_size = 1
total_size = 2000
val_size = 500
scale_rate = 4
verbose = 1
start_index = 0

In [ ]:
x_train_data, y_train_data = importBatch(total_size, start_index, verbose,'train', scale_rate)
from tensorflow.python.keras.utils import to_categorical
y_train_data = to_categorical(y_train_data)
x_train_data = x_train_data.astype('uint8')
y_train_data = y_train_data.astype('uint8')


x_val_data, y_val_data = importBatch(val_size, start_index, verbose, 'val', scale_rate)
y_val_data = to_categorical(y_val_data)
x_val_data = x_val_data.astype('uint8')
y_val_data = y_val_data.astype('uint8')


loaded files input -  100
loaded files input -  200
loaded files input -  300
loaded files input -  400
loaded files input -  500
loaded files input -  600
loaded files input -  700
loaded files input -  800
loaded files input -  900
loaded files input -  1000
loaded files input -  1100
loaded files input -  1200
loaded files input -  1300
loaded files input -  1400


In [12]:
def unet(pretrained_weights=None, input_size=img_shape):
    inputs = Input(input_size)
  
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)    
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))#drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(20, 1, activation = 'sigmoid')(conv9)

    model = Model([inputs], [conv10])

    model.compile(optimizer = Adam(lr = 1e-4), loss = multi_dice_loss, metrics = [multi_dice_loss])

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [13]:
tr_cfg = {
    'hue_delta': 0.2,
    'horisontal_flip': True,
    'crop_rate' : 0.5
}
tr_preprocessing_fn = functools.partial(_augment, **tr_cfg)

In [14]:
val_cfg = {
}
val_preprocessing_fn = functools.partial(_augment, **val_cfg)

In [15]:
train_ds = get_dataset(x_train_data, 
                       y_train_data,
                       preproc_fn=tr_preprocessing_fn,
                       batch_size=1)
val_ds = get_dataset(x_val_data,
                      y_val_data,
                      preproc_fn=val_preprocessing_fn,
                      batch_size=1)


In [18]:
# # TEST #

# temp_ds = get_dataset(x_train_data,
#                      y_train_data,
#                      preproc_fn=tr_preprocessing_fn,
#                      batch_size=10,
#                      shuffle=False)
# data_aug_iter = temp_ds.make_one_shot_iterator()
# next_element = data_aug_iter.get_next()
# for i in range(10):
#     with tf.Session() as sess:
#         batch_of_imgs, batch_of_labels = sess.run(next_element)        
#         print(batch_of_imgs.shape)
#         img = batch_of_imgs[i]
#         label = batch_of_labels[i]
#         label = np.argmax(label, axis=2)        
#         plt.subplot(1, 2, 1)
#         plt.imshow(img)
#         plt.title("Original")
#         plt.subplot(1, 2, 2)
#         plt.imshow(label)
#         plt.title("Masked")
#         plt.show()

In [ ]:
def dice_coefficient(y_true, y_pred):
    smooth = 1.
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f)
                                            + tf.reduce_sum(y_pred_f)
                                            + smooth)
    return score

In [6]:
def multi_dice_loss(y_true, y_pred, alpha=0.5, beta=0.5):
    assert len(y_true.shape) == 4
    assert len(y_pred.shape) == 4
    
    ones = tf.ones(y_true.shape)
    p0 = y_pred
    p1 = ones - y_pred
    g0 = y_true
    g1 = ones - y_true
    
    mult = tf.reduce_sum(tf.multiply(p0, g0), (0, 1, 2))
    den = mult + alpha * tf.reduce_sum(tf.multiply(p0, g1), (0, 1, 2)) + beta * tf.reduce_sum(tf.multiply(p1, g0), (0, 1, 2))
    T = tf.reduce_sum(mult / den)
    max_value = tensorflow.dtypes.cast(y_true.shape[-1], 'float32')
    return max_value - T

In [19]:
print('Starting training...')
# TODO UNCOMMENT PREPROCESSING FUNCTIONS
from tensorflow.keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')

with tf.Session() as sess:
    
    features_placeholder = tf.placeholder(x_train_data.dtype, x_train_data.shape)
    labels_placeholder = tf.placeholder(y_train_data.dtype, y_train_data.shape)
    val_features_placeholder = tf.placeholder(x_val_data.dtype, x_val_data.shape)
    val_labels_placeholder = tf.placeholder(y_val_data.dtype, y_val_data.shape)

    dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
    dataset = dataset.repeat().batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((val_features_placeholder, val_labels_placeholder))
    val_dataset = val_dataset.repeat().batch(batch_size)
    
#     dataset = dataset.map(tr_preprocessing_fn)
#     val_dataset = val_dataset.map(val_preprocessing_fn)
    
    val_iterator = val_dataset.make_initializable_iterator()
    iterator = dataset.make_initializable_iterator()
    
    sess.run(iterator.initializer, feed_dict={features_placeholder: x_train_data,
                                          labels_placeholder: y_train_data})
    sess.run(val_iterator.initializer, feed_dict={val_features_placeholder: x_val_data,
                                                 val_labels_placeholder: y_val_data})
    
#     load_model('rvygon_unet.hdf5')
    model = unet()
    model_checkpoint = ModelCheckpoint('rvygon_unet.hdf5', monitor='loss', verbose=1, save_best_only=True)
    history = model.fit(iterator, steps_per_epoch=2000, 
                        epochs=50,callbacks=[model_checkpoint, csv_logger],
                        validation_data=val_iterator, validation_steps=100)
    

Starting training...
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Epoch 1/50
2000/2000 [==============================] - 96s 48ms/step - loss: 2.7924 - acc: 0.4032 - val_loss: 2.6129 - val_acc: 0.4463

Epoch 00001: loss improved from inf to 2.79241, saving model to rvygon_unet.hdf5
Epoch 2/50
2000/2000 [==============================] - 93s 46ms/step - loss: 2.3216 - acc: 0.4825 - val_loss: 1.7896 - val_acc: 0.2836

Epoch 00002: loss improved from 2.79241 to 2.32163, saving model to rvygon_unet.hdf5
Epoch 3/50
 165/2000 [=>............................] - ETA: 1:23 - loss: 1.5454 - acc: 0.3183

KeyboardInterrupt: 

In [ ]:
# model = unet()
# generator = keras_generator(1,4)
# model_checkpoint = ModelCheckpoint('unet_tensorflow.hdf5', monitor='loss',verbose=1, save_best_only=True)
# history = model.fit_generator(generator,steps_per_epoch=10,epochs=8,callbacks=[model_checkpoint])#, validation_data=(x_val,y_val),validation_steps=20)

In [20]:
SCALE = 4
def eval_model(model):          
        print(1)
        x_pred = model.predict(x_val_data,verbose=1)        
        new_x=np.argmax(x_pred,axis=3)
        
        """ UPSCALING
        new_new_x = np.zeros((new_x.shape[0],new_x.shape[1]*SCALE,new_x.shape[2]*SCALE))
        
        for i in range(x_pred.shape[0]):
            new_new_x[i]=UpscaleImg(new_x[i],SCALE,0)
            upd_print(("Upscaled %d images" % i))
        print()
        new_new_x=new_new_x.astype(int)"""
        
        new_x=new_x.astype(int)        
        
        """   SAVE
        cityscapesPath = os.environ['CITYSCAPES_DATASET']        
        for i in range(len(filenames)):
            impath = os.path.join(cityscapesPath,'results', filenames[i].split('/')[7]+'.png')           
            imsave(impath, new_new_x[i])
            upd_print("Saved %d images" % i)            
        print()
        toc = time.time()
        print('Save files:', toc -tic) """
        y_val = np.argmax(y_val_data,axis=3)
        score = eval_preds(new_x,y_val)        
        return score
x_pred = model.predict(x_val_data[0:1],verbose=1)  
pred_image=np.argmax(x_pred[0],axis=2).astype(int)
pred_image.shape
colorImage(x_val_data[0],pred_image, 'classes.txt', 'colors.txt')

FailedPreconditionError: Error while reading resource variable conv2d/kernel from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/conv2d/kernel)
	 [[Node: conv2d/Conv2D/ReadVariableOp = ReadVariableOp[dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d/kernel)]]
	 [[Node: conv2d_22/Sigmoid/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_332_conv2d_22/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]